In [1]:
!pip install transformers

In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Load the pre-trained vision encoder-decoder model
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [4]:
# Load the feature extractor and tokenizer
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
# Set the device to CUDA if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [6]:
# Set the maximum length and number of beams for caption generation
max_length = 10
num_beams = 4

In [7]:
# Define the generation kwargs
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

In [8]:
# Define the predict_step function to generate captions for input images
def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        # Open and convert the image to RGB if necessary
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)

    # Extract pixel values and move to the specified device
    pixel_values = feature_extractor(images=images, return_tensors='pt').pixel_values
    pixel_values = pixel_values.to(device)

    # Generate caption ids using the model
    with torch.no_grad():
        output_ids = model.generate(pixel_values, **gen_kwargs)

    # Decode the output ids into captions
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return preds

In [9]:
# Example usage
captions = predict_step(["/kaggle/input/pngfile/PNG image"])
print(captions)

['a street sign with a picture of a man']


## The image captioning process involves two main steps: image feature extraction and caption generation.

For image feature extraction, the project utilizes the ViT (Vision Transformer) model. The ViT model is a state-of-the-art deep learning model designed for image classification and feature extraction. It transforms the input image into a sequence of patches and passes them through a series of Transformer layers to capture the visual information. The ViT model used in this project has been pre-trained on a large dataset and can extract meaningful visual features from images.

Once the image features are extracted, the project uses the GPT2 (Generative Pre-trained Transformer 2) model for caption generation. The GPT2 model is a powerful language model capable of generating coherent and contextually relevant text. In this project, the GPT2 model is fine-tuned on a large dataset of image-caption pairs to learn the association between visual features and textual descriptions.

## To generate captions for input images, the project follows the following steps:

1.Load the pre-trained vision encoder-decoder model, which combines the ViT model for image feature extraction and the GPT2 model for      caption generation.

2.Load the feature extractor and tokenizer. The feature extractor converts the input image into a format suitable for the model, and the tokenizer converts the generated caption from token IDs to human-readable text.

3.Set the device to CUDA if available (to leverage GPU acceleration), otherwise use CPU.

4.Define the maximum length and number of beams for caption generation. The maximum length limits the number of tokens in the generated caption, and the number of beams controls the diversity of generated captions.

5.Define the generation kwargs, which include the maximum length and number of beams.

6.Define the predict_step function, which takes a list of image paths as input and performs the following steps:

a.Open and convert each image to RGB if necessary.

b.Extract pixel values from the images and move them to the specified device.

c.Generate caption IDs using the model and the extracted image features.

d.Decode the output IDs into captions using the tokenizer.

e.Return the generated captions as a list.

f.Call the predict_step function with the file paths of the input images to generate captions.

7.Call the predict_step function with the file paths of the input images to generate captions.

## Overall, this image captioning project leverages the power of the ViT-GPT2 model to extract visual features from images and generate    descriptive captions. It can be used in various applications such as image indexing, content-based image retrieval, and enhancing accessibility for visually impaired individuals